### Imports

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
 
from sklearn.feature_extraction.text import TfidfVectorizer
 
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline

### Loading our Data

In [2]:
con = sqlite3.connect("RecipeDB.db")
#cur = con.cursor();

sql = "SELECT * FROM Recipe ;"
df = pd.read_sql(sql,con)
df.index = df['recID']
df = df.drop(['recID'],axis = 1)

## A glance of the Data

In [3]:
df.head()

,recName,recHeadline,recDisc,recDifficulty,recServingSize,recLink,recRating,recRatingCount,recSteps,recTime
recID,,,,,,,,,,
53f45a29ff604dac328b456e,Pikante Paprika-Bratwurst Pfanne,auf Petersilien-Couscous,Dieses Gericht zaubert Dir nicht nur optisch A...,2,0,https://ddw4dkk7s1lkt.cloudfront.net/card/53f4...,3.1,765,6,35
5469feafff604d2c8b8b4569,Gnocchi-Paprika-Mozzarella Auflauf,verfeinert mit frischem Basilikum,"Tu Dir was Gutes! Unser Soul Food hilft Dir, s...",1,600,https://ddw4dkk7s1lkt.cloudfront.net/card/gnoc...,2.8,1012,6,30
54c91e3e6ced6e5f178b4571,Steak mit gebratenen Topinambur-Kartoffelecken,Rosmarin und Lauchcremesoße,"Topinambur kann man roh oder gegart essen, und...",2,0,https://ddw4dkk7s1lkt.cloudfront.net/card/54c9...,3.3,1186,7,40
54c91e716ced6e39178b456c,Hähnchenoberkeule auf Ofengemüse,verfeinert mit Sesam und Erdnusssoße,Kohlenhydratarm und proteinreich! Vor allem Pr...,2,0,https://ddw4dkk7s1lkt.cloudfront.net/card/54c9...,3.3,1194,7,30
54c91ed56ced6e5f178b4572,Valentinstagsquiche,"mit zweifarbigen Karotten, Zucchini und Rucola...",Der Valentinstag! Für viele wohl eine Erfindun...,3,0,https://ddw4dkk7s1lkt.cloudfront.net/card/54c9...,2.6,1407,8,30


In [25]:
df2 = df.assign(recString = df.recName.astype(str) + " " + df.recHeadline.astype(str) + " " +  df.recDisc.astype(str))
df2 = df2[["recString"]]
df2

,recString
recID,
53f45a29ff604dac328b456e,Pikante Paprika-Bratwurst Pfanne auf Petersili...
5469feafff604d2c8b8b4569,Gnocchi-Paprika-Mozzarella Auflauf verfeinert ...
54c91e3e6ced6e5f178b4571,Steak mit gebratenen Topinambur-Kartoffelecken...
54c91e716ced6e39178b456c,Hähnchenoberkeule auf Ofengemüse verfeinert mi...
54c91ed56ced6e5f178b4572,"Valentinstagsquiche mit zweifarbigen Karotten,..."
...,...
5c30b8a7c445fa33f45d3002,"Mildes gelbes Curry mit Limette, Kichererbsen ..."
5c30bb90c445fa36747c59b2,Hello Mole! Feuriges Chili con Carne mit gebac...
5c30c098e3f33962d2048221,3-Gänge-Valentinstags Menü mit Auberginen-Carp...


In [26]:
print(df2.iloc[0]["recString"])

Pikante Paprika-Bratwurst Pfanne auf Petersilien-Couscous Dieses Gericht zaubert Dir nicht nur optisch Abwechslung auf Deinen Teller, sondern überzeugt auch auf geschmacklicher Ebene voll und ganz! Die Bratwurst sorgt für eine pikant-würzige Note, Paprika und Petersilie bringen Frische und Farbe in das Gericht. Lass es Dir schmecken!


In [4]:
df.shape

(1897, 10)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1897 entries, 53f45a29ff604dac328b456e to 5c30c44ec445fa3dbb5a80b2
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   recName         1897 non-null   object 
 1   recHeadline     1897 non-null   object 
 2   recDisc         1897 non-null   object 
 3   recDifficulty   1897 non-null   int64  
 4   recServingSize  1897 non-null   int64  
 5   recLink         1897 non-null   object 
 6   recRating       1897 non-null   float64
 7   recRatingCount  1897 non-null   int64  
 8   recSteps        1897 non-null   int64  
 9   recTime         1897 non-null   int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 243.0+ KB


In [5]:
df.describe()

,recDifficulty,recServingSize,recRating,recRatingCount,recSteps,recTime
count,1897.000000,1897.000000,1897.000000,1897.000000,1897.000000,1897.000000
mean,1.672114,351.470743,3.024829,491.638904,5.946231,38.483922
std,0.621373,331.467402,0.868404,488.070676,1.031803,16.209106
min,0.000000,0.000000,0.000000,0.000000,1.000000,10.000000
25%,1.000000,0.000000,3.100000,80.000000,6.000000,30.000000
50%,2.000000,450.000000,3.300000,351.000000,6.000000,30.000000
75%,2.000000,650.000000,3.400000,742.000000,6.000000,40.000000
max,3.000000,1500.000000,3.800000,3049.000000,10.000000,180.000000


In [6]:
test_df = df

#func to normalize text
def process_text(text):
    #replace muliple spaces with one
    test = ''.join(text.split())
    #lowercase
    text = text.lower()
    return text

test_df['recDisc'] = test_df.apply(lambda x: process_text(x.recDisc),axis= 1)

tf_idf = TfidfVectorizer()
t_matrix = tf_idf.fit_transform(test_df['recDisc'])

In [9]:
print(t_matrix)

  (0, 5319)	0.0779096427291905
  (0, 1582)	0.06645175881451644
  (0, 3746)	0.07936367275496509
  (0, 1099)	0.07262314745099409
  (0, 3079)	0.07315521283941849
  (0, 1668)	0.18051302408037395
  (0, 1924)	0.14665561614474942
  (0, 845)	0.15691809604294546
  (0, 4668)	0.2262698775168333
  (0, 4595)	0.1630209069626895
  (0, 4411)	0.15814896946163465
  (0, 6957)	0.16446069857580528
  (0, 4712)	0.2198357683794907
  (0, 1313)	0.0913074952740633
  (0, 2041)	0.08158972714202938
  (0, 5660)	0.14113600985367597
  (0, 823)	0.2472774000819999
  (0, 1175)	0.059151528760695224
  (0, 2053)	0.1136943781511668
  (0, 6249)	0.12800551306251115
  (0, 6564)	0.19768063195517796
  (0, 1270)	0.3109948957016076
  (0, 2366)	0.3109948957016076
  (0, 327)	0.09024359500834243
  (0, 7243)	0.23395649587227654
  :	:
  (1896, 3074)	0.11444755640115359
  (1896, 6786)	0.12164951994243137
  (1896, 5575)	0.05341797582210556
  (1896, 1138)	0.08573854635171084
  (1896, 5584)	0.09851196700282386
  (1896, 2182)	0.0863672965066

In [11]:
csm = cosine_similarity(t_matrix,t_matrix)
csm.shape


(1897, 1897)